In [1]:
import $file.hw7stdlib
import hw7stdlib._

Compiling /home/user/Downloads/Assignment7/hw7stdlib.sc

import $file.$        

import hw7stdlib._

# Homework 7

In this assignment we will implement functions and recursive let bindings to support recursive functions.

## Additional Syntax (10 pts)

Add the syntax as case classes necessary to implement functions. Below is the grammar to get you started along with the syntax we have defined so far:

$$\begin{array}{rcll}
\mathbf{Expr} & ::=\ & ... \\
 & | & FunDef(\textbf{String}, \textbf{Expr}) \\ 
 & | & FunCall(\textbf{Expr}, \textbf{Expr}) \\
 & | & LetRec(\textbf{String}, \textbf{String}, \textbf{Expr}, \textbf{Expr}) \\
\end{array}$$

**Note:** `Const` has been renamed to `Num`

In [2]:
sealed trait Expr
case class Num(x : Integer) extends Expr
case class Bin(x : Bool) extends Expr
case class Ident(x : String) extends Expr
case class Plus(x : Expr, y : Expr) extends Expr
case class Minus(x : Expr, y : Expr) extends Expr
case class Mult(x : Expr, y : Expr) extends Expr
case class Pow(x : Expr, y : Expr) extends Expr
case class Neg(x : Expr) extends Expr
case class Eq(x : Expr, y : Expr) extends Expr
case class And(x : Expr, y : Expr) extends Expr
case class Or(x : Expr, y : Expr) extends Expr
case class IfThenElse(p : Expr, t : Expr, f : Expr) extends Expr
case class Let(id : String, x : Expr, y : Expr) extends Expr
case class FunDef(x: String, y: Expr) extends Expr
case class FunCall(x: Expr, y: Expr) extends Expr
case class LetRec(x: String, y: String, e1: Expr, e2: Expr) extends Expr

defined trait Expr
defined class Num
defined class Bin
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Pow
defined class Neg
defined class Eq
defined class And
defined class Or
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class LetRec

In [3]:
val f = FunDef("x", Ident("x"))
passed(4)


*** Tests Passed (4 points) ***


f: FunDef = FunDef("x", Ident("x"))

In [4]:
val call = FunCall(f, Num(Positive(three)))
passed(3)


*** Tests Passed (3 points) ***


call: FunCall = FunCall(
  FunDef("x", Ident("x")),
  Num(Positive(Succ(Succ(Succ(Zero)))))
)

In [5]:
val lrec = LetRec("f", "x", Plus(Ident("x"), Num(Positive(three))), call)
passed(3)


*** Tests Passed (3 points) ***


lrec: LetRec = LetRec(
  "f",
  "x",
  Plus(Ident("x"), Num(Positive(Succ(Succ(Succ(Zero)))))),
  FunCall(FunDef("x", Ident("x")), Num(Positive(Succ(Succ(Succ(Zero))))))
)

## Closures (5 pts)

Extend the defininition of $\mathbf{Value}$ to support closures:
$$\begin{array}{rcll}
\mathbf{Value} & ::=\ & ... \\
 & | & Closure(\textbf{String}, \textbf{Expr}, \textbf{Environment}) \\
\end{array}$$

### Note: 
This cell also contains the new definition for environments needed for recursion. It differs from using dictionaries in that:

1. it is non-polymorphic
2. The extend-rec constructor will be used to create closures for a let-rec binding. Where f is the key/name of the function and then x, e, and sigma represent the elements in the closure. We implement this functionality in the lookup function for environments

**When you implement the case for LetRec below you will want to use ExtendRec to extend the environment**

The following mathematical notation:

$$\sigma' = \sigma[x_\text{name} \mapsto \texttt{Closure}(x_\text{param}, e_\text{body}, \sigma')]$$

would be this in scala using the new definition:

```scala
val sigma_prime = ExtendRec(x_name, x_param, e_body, sigma)
```

In [7]:
// Values
sealed trait Value
case class NumVal(x : Integer) extends Value
case class BinVal(x : Bool) extends Value
case object Error extends Value
case class Closure(x: String, y: Expr, z: Environment) extends Value

// New definition for environment for recursive funcs:
sealed trait Environment
case object EmptyEnv extends Environment 
case class Extend(k : String, v : Value, env : Environment) extends Environment
case class ExtendRec(f: String, x: String, e: Expr, sigma: Environment ) extends Environment

defined trait Value
defined class NumVal
defined class BinVal
defined object Error
defined class Closure
defined trait Environment
defined object EmptyEnv
defined class Extend
defined class ExtendRec

In [8]:
val c = Closure("x", Ident("x"), EmptyEnv)
passed(5)


*** Tests Passed (5 points) ***


c: Closure = Closure("x", Ident("x"), EmptyEnv)

### New lookup function
This function is used instead of the one for dictionaries when working with environments. You don't need to use it as the Ident case is filled out already.

In [9]:
def lookupEnv(sigma: Environment, x: String): Maybe[Value] = sigma match {
    case EmptyEnv => None
    case Extend(y, v, pi) => string_eq(y,x) match {
        case True => Just(v)
        case False => lookupEnv(pi, x)
    } 
    case ExtendRec(f, y, e, pi) => string_eq(x,f) match {
        case True => Just(Closure(y, e, sigma))
        case False => lookupEnv(pi, x)
    } 
}   

defined function lookupEnv

## Updating the interpreter (20 pts)

Add new cases to the interpreter to handle the new expressions we have added to the language. Once this is done you should be able to support both recursive and non-recursive functions. We have given you the skeleton to get started.

In [60]:
def eval(env : Environment, expr : Expr) : Value = expr match {
    case Num(n)    => NumVal(n)
    case Bin(p)    => BinVal(p)
    case Ident(id) => lookupEnv(env, id) match {
        case Just(v) => v
        case None    => Error
    }
    case Plus(e1, e2)  => eval_bin_arith(plus, env, e1, e2)
    case Minus(e1, e2) => eval_bin_arith(minus, env, e1, e2)
    case Mult(e1, e2)  => eval_bin_arith(mult, env, e1, e2)
    case Pow(e1, e2)   => (eval(env, e1), eval(env,e2)) match {
        case (NumVal(x), NumVal(Positive(n))) => NumVal(pow(x, n))
        case _ => Error
    }
    case Neg(e) => eval(env, e) match {
        case NumVal(x) => NumVal(negate(x))
        case BinVal(p) => BinVal(not(p))
        case Error     => Error
    }
    case Eq(e1, e2) => (eval(env, e1), eval(env, e2)) match {
        case (NumVal(x), NumVal(y)) => BinVal(int_eq(x,y))
        case (BinVal(p), BinVal(q)) => BinVal(bool_eq(p,q))
        case _                      => Error
    }
    case And(e1, e2) => eval_bin_bool(and, env, e1, e2)
    case Or(e1, e2)  => eval_bin_bool(or, env, e1, e2)
    case IfThenElse(p, e_t, e_f) => eval(env, p) match{
        case BinVal(True)  => eval(env, e_t)
        case BinVal(False) => eval(env, e_f)
        case _             => Error
    }
    case Let(id, df, body) => eval(env, df) match{
        case Error => Error
        case x     => {
            val new_env = Extend(id, x, env)
            eval(new_env , body)
        }
    }
    
    case FunDef(x, e_body) => Closure(x, e_body, env)
    
    case FunCall(e_f, e_arg) => eval(env, e_f) match
    {
        case Closure(x, ex, env1) => eval(env, e_arg) match
        {
            case Error => Error
            case x1 => {eval(Extend(x, x1, env1), ex)}
        }
        case _ => Error
    }
        
    
    case LetRec(x_name, x_param, e_body, e_in) => eval(ExtendRec(x_name, x_param, e_body, env), e_in) 
        
}

def eval_bin_arith( op : (Integer, Integer) => Integer
                  , env : Environment
                  , e1 : Expr
                  , e2 : Expr) : Value 
    = (eval(env, e1), eval(env, e2)) match{
        case (NumVal(x), NumVal(y)) => NumVal(op(x,y))
        case _ => Error
    }

def eval_bin_bool( op : (Bool, Bool) => Bool
                 , env : Environment
                 , e1 : Expr
                 , e2 : Expr) : Value 
    = (eval(env, e1), eval(env, e2)) match{
        case (BinVal(x), BinVal(y)) => BinVal(op(x,y))
        case _ => Error
    }

defined function eval
defined function eval_bin_arith
defined function eval_bin_bool

In [54]:
//
// Tests Function definition
//

val x = Ident("x")
val f = Ident("f")

// function(x) x
val e_1 = FunDef("x", x)
assert(eval(EmptyEnv, e_1) == Closure("x", x, EmptyEnv))
passed(5)


*** Tests Passed (5 points) ***


x: Ident = Ident("x")
f: Ident = Ident("f")
e_1: FunDef = FunDef("x", Ident("x"))

In [55]:
//
// Tests Function Calls
//

// (function(x) x)(2)
val e_2 = FunCall(e_1, Num(Positive(two)))
assert(eval(EmptyEnv, e_2) == NumVal(Positive(two)))
passed(5)


*** Tests Passed (5 points) ***


e_2: FunCall = FunCall(FunDef("x", Ident("x")), Num(Positive(Succ(Succ(Zero)))))

In [56]:
//
// Tests Closed environments
//

// let y = 3 in
//   let f = function(x) y in
//     let y = 4 in
//       f(0)
val e_3 =
    Let("y", Num(Positive(three)),
        Let("f", FunDef("x", Ident("y")),
            Let("y", Num(Positive(four)),
                FunCall(f, Num(Positive(Zero)))
            )
        )
    )
assert(eval(EmptyEnv, e_3) == NumVal(Positive(three)))
passed(5)


*** Tests Passed (5 points) ***


e_3: Let = Let(
  "y",
  Num(Positive(Succ(Succ(Succ(Zero))))),
  Let(
    "f",
    FunDef("x", Ident("y")),
    Let(
      "y",
      Num(Positive(Succ(Succ(Succ(Succ(Zero)))))),
      FunCall(Ident("f"), Num(Positive(Zero)))
    )
  )
)

In [61]:
//
// Tests Recursion
//

// if x == 0 then 1 else x * f(x - 1)
val factorial_body = IfThenElse(
    Eq(x, Num(Positive(Zero))),
    Num(Positive(one)),
    Mult(x, FunCall(f, Minus(x, Num(Positive(one)))))
)

// let rec f = function(x) if x == 0 then 1 else x * f(x - 1)
//     in f(3)
val e_1 = LetRec("f", "x", factorial_body, FunCall(f, Num(Positive(three))))

assert(eval(EmptyEnv, e_1) == NumVal(Positive(six)))
passed(5)


*** Tests Passed (5 points) ***


factorial_body: IfThenElse = IfThenElse(
  Eq(Ident("x"), Num(Positive(Zero))),
  Num(Positive(Succ(Zero))),
  Mult(
    Ident("x"),
    FunCall(Ident("f"), Minus(Ident("x"), Num(Positive(Succ(Zero)))))
  )
)
e_1: LetRec = LetRec(
  "f",
  "x",
  IfThenElse(
    Eq(Ident("x"), Num(Positive(Zero))),
    Num(Positive(Succ(Zero))),
    Mult(
      Ident("x"),
      FunCall(Ident("f"), Minus(Ident("x"), Num(Positive(Succ(Zero)))))
    )
  ),
  FunCall(Ident("f"), Num(Positive(Succ(Succ(Succ(Zero))))))
)